## Dataset preparation
I will use dataset from Kaggle, so first of all I will install kaggle library.

In [0]:
!pip install kaggle

In [0]:
!mkdir /content/.kaggle

In order to use kaggle library (wraper for Kaggle API) you need to get API credentials. To do that:
1. That login to your Kaggle account at https://www.kaggle.com
2. Go to the "My Account" tab of your user profile
3. Find API block and click "Create New API Token"

In [0]:
import json
token = {"username":"{YOUR_USERNAME}","key":"{YOUR_KEY}"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

Lets search for datasets by keyword "hotel"

In [0]:
!kaggle datasets list -s hotel

I am going to mount my Google drive in order to save model checkpoints

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd /content/drive/My\ Drive/gpt2

Lets download dataset that will be used for training - 515k-hotel-reviews-data-in-europe

In [0]:
!kaggle datasets download -d jiashenliu/515k-hotel-reviews-data-in-europe -p /content/drive/My\ Drive/gpt2/training_data

In [0]:
!unzip /content/drive/My\ Drive/gpt2/training_data/515k-hotel-reviews-data-in-europe.zip -d /content/drive/My\ Drive/gpt2/training_data

In [0]:
!ls -la /content/drive/My\ Drive/gpt2/training_data
!ls -la /content/drive/My\ Drive/gpt2/

Dataset contains both positive and negative reviews, I will merge those reviews into single column

In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/gpt2/training_data/Hotel_Reviews.csv')
df = pd.concat([df['Negative_Review'], df['Positive_Review']], ignore_index=True)
df = df.loc[~df.isin(['No Negative', 'No Positive'])]
df = df.loc[df.str.len() > 20]
df = df.drop_duplicates()


df.to_csv('/content/drive/My Drive/gpt2/training_data/Hotel_Reviews_Training.csv', header=False, index=False)

## Preparing for training

GPT-2 cloning

In [0]:
!git clone https://github.com/nshepperd/gpt-2.git

In [0]:
cd gpt-2

Requirements installing

In [0]:
!pip3 install -r requirements.txt

Downloading base 117M model

In [0]:
!python3 download_model.py 117M

In [0]:
!export PYTHONIOENCODING=UTF-8

Preparing dataset for training: precomputing dataset's encodings

In [0]:
!PYTHONPATH=src python3 encode.py /content/drive/My\ Drive/gpt2/training_data/Hotel_Reviews_Training.csv /content/drive/My\ Drive/gpt2/training_data/Hotel_Reviews_Training.csv.npz

## Training model on hotel reviews dataset

Every checkpoint will be saved in your Google drive My Drive/gpt2/gpt-2/checkpoint/

In [0]:
!PYTHONPATH=src python3 train.py --dataset /content/drive/My\ Drive/gpt2/training_data/Hotel_Reviews_Training.csv.npz